<a href="https://colab.research.google.com/github/86HenriqueSilva/mega_sena/blob/main/05_MEGA_SENA_TURBO_v_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previsão de Números da Mega Sena com Rede Neural e Análise de Atrasos

Este notebook utiliza uma rede neural para prever números da Mega Sena com base em dados históricos e calcula os atrasos das dezenas para identificar quais estão mais tempo sem aparecer. Abaixo está uma explicação detalhada de cada parte do código:

## 1. Importação de Bibliotecas
Carregamos as bibliotecas necessárias para manipulação de dados, treinamento do modelo, e visualização do progresso.

```python
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from itertools import combinations
from tqdm.notebook import tqdm

2. Função para Cálculo de Frequências

Calcula a frequência dos números sorteados em cada coluna dos dados.

python

def calcular_frequencias(df):
    frequencias = {}
    for coluna in ['Bola 1', 'Bola 2', 'Bola 3', 'Bola 4', 'Bola 5', 'Bola 6']:
        bolas = df[coluna].value_counts().sort_values(ascending=True)
        frequencias[coluna] = bolas
    return frequencias

3. Função para Pós-processamento das Previsões

Garante que as previsões contenham números únicos para cada bola.

python

def postprocess_predictions(predictions):
    counts = np.bincount(predictions, minlength=61)
    for i in range(len(predictions)):
        while counts[predictions[i]] > 1:
            new_value = np.random.randint(1, 61)
            while counts[new_value] > 0:
                new_value = np.random.randint(1, 61)
            predictions[i] = new_value
            counts[predictions[i]] += 1
    return predictions

4. Carregamento dos Dados

Baixa e carrega o arquivo CSV com dados históricos da Mega Sena. Remove linhas inválidas e a coluna de data, se presente.

python

data_url = 'https://drive.google.com/uc?id=1t28TCezmfyBc5X_3-JwqN-FxkV63M-t5'
data = pd.read_csv(data_url)
data = data.dropna()
data = data.drop(columns=['Data'])

5. Cálculo dos Atrasos das Dezenas

Calcula o atraso de cada número (número de concursos desde a última vez que o número foi sorteado).

python

total_concursos = len(data)
atrasos = {f'Bola {i}': {num: None for num in range(1, 61)} for i in range(1, 7)}
for index, row in data.iterrows():
    for coluna in atrasos:
        numero = row[coluna]
        atrasos[coluna][numero] = index

for coluna in atrasos:
    for numero in atrasos[coluna]:
        if atrasos[coluna][numero] is None:
            atrasos[coluna][numero] = total_concursos
        else:
            atrasos[coluna][numero] = total_concursos - atrasos[coluna][numero]

6. Criação e Filtragem dos Dados de Atrasos

Cria um DataFrame com os atrasos e filtra as 10 dezenas com maiores atrasos em cada coluna.

python

atrasos_data = []
for coluna, atrasos_coluna in atrasos.items():
    for numero, atraso in atrasos_coluna.items():
        atrasos_data.append({'Coluna': coluna, 'Numero': numero, 'Atraso': atraso})

atrasos_df = pd.DataFrame(atrasos_data)
top_10_atrasos = []
for coluna in atrasos_df['Coluna'].unique():
    top_10_atrasos.extend(atrasos_df[atrasos_df['Coluna'] == coluna].nlargest(10, 'Atraso')['Numero'].tolist())

top_10_atrasos_unicos = sorted(set(top_10_atrasos))
mascara = data[colunas_bolas].isin(top_10_atrasos_unicos).all(axis=1)
data_filtrada = data[mascara]

7. Pré-processamento dos Dados

Prepara os dados para treinamento da rede neural, normalizando as características.

python

colunas_bolas = ['Bola 1', 'Bola 2', 'Bola 3', 'Bola 4', 'Bola 5', 'Bola 6']
X = data_filtrada.drop(columns=colunas_bolas)
y = data_filtrada[colunas_bolas]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=512)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

8. Construção e Treinamento do Modelo

Cria e treina uma rede neural para prever os números dos sorteios da Mega Sena.

python

model = tf.keras.Sequential([
    tf.keras.layers.Dense(1024, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(6)
])
model.compile(optimizer='adam', loss='mean_squared_error')

epochs = 15000
batch_size = 256
with tqdm(total=epochs, desc="Treinamento da Rede Neural") as pbar:
    for epoch in range(epochs):
        pbar.update(1)
        model.fit(X_train_scaled, y_train, batch_size=batch_size, verbose=0)
        pbar.set_postfix({'Epoch': epoch + 1})

9. Avaliação do Modelo

Avalia o desempenho do modelo no conjunto de teste e exibe o erro médio absoluto.

python

y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)

10. Exibição das Previsões

Mostra as previsões ajustadas para o próximo sorteio.

python

y_pred_int = np.clip(np.round(y_pred), 1, 60).astype(int)
y_pred_int_postprocessed = postprocess_predictions(y_pred_int[0])

print("Previsões pós-processadas para o próximo sorteio:")
for i, pred in enumerate(y_pred_int_postprocessed):
    print(f"Bola {i+1}: {pred}")

print("Erro médio absoluto no conjunto de teste: {:.2f}".format(mae))

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from itertools import combinations
from tqdm.notebook import tqdm

# Função para cálculo de frequências
def calcular_frequencias(df):
    frequencias = {}
    for coluna in ['Bola 1', 'Bola 2', 'Bola 3', 'Bola 4', 'Bola 5', 'Bola 6']:
        bolas = df[coluna].value_counts().sort_values(ascending=True)
        frequencias[coluna] = bolas
    return frequencias

# Função para pós-processamento das previsões
def postprocess_predictions(predictions):
    counts = np.bincount(predictions, minlength=61)
    for i in range(len(predictions)):
        while counts[predictions[i]] > 1:
            new_value = np.random.randint(1, 61)
            while counts[new_value] > 0:
                new_value = np.random.randint(1, 61)
            predictions[i] = new_value
            counts[predictions[i]] += 1
    return predictions

# URL do arquivo CSV
data_url = 'https://drive.google.com/uc?id=1t28TCezmfyBc5X_3-JwqN-FxkV63M-t5'

# Carregar o arquivo CSV em um DataFrame
data = pd.read_csv(data_url)

# Remover linhas com valores não numéricos
data = data.dropna()

# Remover a coluna de data
data = data.drop(columns=['Data'])

# Calcular os atrasos das dezenas
total_concursos = len(data)
atrasos = {f'Bola {i}': {num: None for num in range(1, 61)} for i in range(1, 7)}
for index, row in data.iterrows():
    for coluna in atrasos:
        numero = row[coluna]
        atrasos[coluna][numero] = index

for coluna in atrasos:
    for numero in atrasos[coluna]:
        if atrasos[coluna][numero] is None:
            atrasos[coluna][numero] = total_concursos
        else:
            atrasos[coluna][numero] = total_concursos - atrasos[coluna][numero]

# Criar um DataFrame temporário para armazenar os atrasos
atrasos_data = []
for coluna, atrasos_coluna in atrasos.items():
    for numero, atraso in atrasos_coluna.items():
        atrasos_data.append({'Coluna': coluna, 'Numero': numero, 'Atraso': atraso})

atrasos_df = pd.DataFrame(atrasos_data)

# Filtrar as 10 dezenas com maiores atrasos em cada coluna
top_10_atrasos = []
for coluna in atrasos_df['Coluna'].unique():
    top_10_atrasos.extend(atrasos_df[atrasos_df['Coluna'] == coluna].nlargest(10, 'Atraso')['Numero'].tolist())

# Remover duplicatas e ordenar os números
top_10_atrasos_unicos = sorted(set(top_10_atrasos))

# Criar uma máscara para filtrar as colunas do DataFrame
colunas_bolas = ['Bola 1', 'Bola 2', 'Bola 3', 'Bola 4', 'Bola 5', 'Bola 6']
mascara = data[colunas_bolas].isin(top_10_atrasos_unicos).all(axis=1)
data_filtrada = data[mascara]

# Pré-processamento dos dados
X = data_filtrada.drop(columns=colunas_bolas)
y = data_filtrada[colunas_bolas]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=512)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Construir e treinar o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1024, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(6)  # 6 saídas para prever os 6 números
])
model.compile(optimizer='adam', loss='mean_squared_error')

epochs = 15000
batch_size = 256
with tqdm(total=epochs, desc="Treinamento da Rede Neural") as pbar:
    for epoch in range(epochs):
        pbar.update(1)
        model.fit(X_train_scaled, y_train, batch_size=batch_size, verbose=0)
        pbar.set_postfix({'Epoch': epoch + 1})

# Avaliar o modelo
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)

# Arredondar as previsões para valores inteiros dentro do intervalo de 1 a 60
y_pred_int = np.clip(np.round(y_pred), 1, 60).astype(int)

# Postprocessamento das previsões
y_pred_int_postprocessed = postprocess_predictions(y_pred_int[0])

# Exibir as previsões pós-processadas para o próximo concurso
print("Previsões pós-processadas para o próximo sorteio:")
for i, pred in enumerate(y_pred_int_postprocessed):
    print(f"Bola {i+1}: {pred}")

# Calcular e exibir o erro médio absoluto no conjunto de teste com duas casas decimais
print("Erro médio absoluto no conjunto de teste: {:.2f}".format(mae))


Treinamento da Rede Neural:   0%|          | 0/15000 [00:00<?, ?it/s]

2/2 [==============================] - 0s 28ms/step
Previsões pós-processadas para o próximo sorteio:
Bola 1: 22
Bola 2: 19
Bola 3: 42
Bola 4: 33
Bola 5: 40
Bola 6: 11
Erro médio absoluto no conjunto de teste: 15.95
